In [48]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator


In [49]:
from keras.callbacks import ModelCheckpoint

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout

In [51]:
img_size = (28, 28, 1)
layer_sizes = (128, 64)
n_filters = (32, 64, 128)
kernel_sizes = (7, 3, 3)
pool_size = 2
drop_prob = 0.5


n_classes = 345   #output_dim/units is equal to the number of image classes

In [52]:
model = Sequential([Convolution2D(filters = 32,
                        kernel_size = (3,3),
                        activation = "relu",
                        input_shape = img_size),
                    Convolution2D(filters = 32,
                        kernel_size = (3,3),
                        activation = "relu"),
                    MaxPooling2D(pool_size=(2,2)),
                    Dropout(.20),
                    Convolution2D(filters = 64,
                        kernel_size = (3,3),
                        activation = "relu"),
                    Convolution2D(filters = 64,
                        kernel_size = (3,3),
                        activation = "relu"),
                    MaxPooling2D(pool_size=(2,2)),
                    Dropout(.1),
                    Flatten(),
                    Dense(256, activation="relu"),
                    Dense(n_classes, activation="softmax") 
                   ])

In [53]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 10, 10, 64)        18496     
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 8, 8, 64)          36928     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 4, 4, 64)         

### Generating additional training and test data using ImageDataGenerator

In [54]:
train_path = "datasets/data/train_set/"
test_path = "datasets/data/test_set/"

data_file_path = "datasets/data/numpy_bitmap"

In [55]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=False,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


In [56]:
%%time

training_set = train_datagen.flow_from_directory(
        train_path,
        target_size=(28, 28),
        color_mode="grayscale",
        batch_size=128,
        class_mode='categorical')



Found 2760000 images belonging to 345 classes.
Wall time: 1min 43s


In [57]:
%%time

test_set = test_datagen.flow_from_directory(
        test_path,
        target_size=(28, 28),
        color_mode="grayscale",
        batch_size=128,
        class_mode='categorical') 



Found 690000 images belonging to 345 classes.
Wall time: 30.5 s


### Training the model

In [58]:
model.compile(optimizer = "nadam", loss="categorical_crossentropy", metrics=["accuracy"])

In [59]:
save_cb = ModelCheckpoint("conv_doodle_model.h5",
                          save_best_only=True)

In [ ]:
%%time

history = model.fit_generator(
            training_set,
            steps_per_epoch=2760000,
            epochs=5,
            validation_data=test_set,
            validation_steps=690000, 
            callbacks=[save_cb])



Epoch 1/5
  31970/2760000 [..............................] - ETA: 279:44:04 - loss: 3.0746 - accuracy: 0.3189

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1);

### Saving model

In [ ]:
model.save("conv_doodle_model.h5")


In [ ]:
model_json = model.to_json()
with open("conv_doodle_model_json.json", "w") as json_file:
    json_file.write(model_json)

In [16]:
# convolution layer 1
#model.add(Convolution2D(filters = 32,
#                        kernel_size = (3,3),
#                        activation = "relu",
#                        input_shape = (28,28,1)))


In [ ]:
# convolution layer 2
#model.add(Convolution2D(filters = 32,
#                        kernel_size = (3,3),
#                        activation = "relu"))


In [ ]:
# maxpooling layer 1
#model.add(MaxPooling2D(pool_size=(2,2)))


In [ ]:
# deactivating some elements to avoid overfitting
#model.add(Dropout(.20))


In [ ]:
# convolution layer 3
#model.add(Convolution2D(filters = 64,
#                        kernel_size = (3,3),
#                        activation = "relu"))
                        

In [ ]:
# convolution layer 4
#model.add(Convolution2D(filters = 64,
#                        kernel_size = (3,3),
#                        activation = "relu"))


In [ ]:
# maxpooling layer 2
#model.add(MaxPooling2D(pool_size=(2,2)))


In [ ]:
# deactivating some elements
#model.add(Dropout(.1))


In [ ]:
# flattening layer
#model.add(Flatten())

# fully connected layer
#model.add(Dense(256, activation="relu"))
#model.add(Dense(output_dim=5, activation="softmax"))  #output_dim is the number of classes used for classification
                                                      # i.e the image categories  


